In [1]:
import numpy as np

import mdctn

#### MDCT4 by smagt

Just using this for comparision. These functions were incredibly useful for validating my mdct works. Thanks smagt!

https://github.com/smagt/mdct

In [2]:
#mdct4 by smagt: https://github.com/smagt/mdct
def mdct4(x):
    N = x.shape[0]
    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")
    M = N // 2
    N4 = N // 4
    
    rot = np.roll(x, N4)
    rot[:N4] = -rot[:N4]
    t = np.arange(0, N4)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N)
    c = np.take(rot,2*t) - np.take(rot, N-2*t-1) \
        - 1j * (np.take(rot, M+2*t) - np.take(rot,M-2*t-1))
    c = (2./np.sqrt(N)) * w * np.fft.fft(0.5 * c * w, N4)
    y = np.zeros(M)
    y[2*t] = np.real(c[t])
    y[M-2*t-1] = -np.imag(c[t])
    return y


def imdct4(x):
    N = x.shape[0]
    if N%2 != 0:
        raise ValueError("iMDCT4 only defined for even-length vectors.")
    M = N // 2
    N2 = N*2
    
    t = np.arange(0,M)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N2)
    c = np.take(x,2*t) + 1j * np.take(x,N-2*t-1)
    c = 0.5 * w * c
    c = np.fft.fft(c,M)
    c = ((8 / np.sqrt(N2))*w)*c
    
    rot = np.zeros(N2)
    
    rot[2*t] = np.real(c[t])
    rot[N+2*t] = np.imag(c[t])
    
    t = np.arange(1,N2,2)
    rot[t] = -rot[N2-t-1]
    
    t = np.arange(0,3*M)
    y = np.zeros(N2)
    y[t] = rot[t+M]
    t = np.arange(3*M,N2)
    y[t] = -rot[t-3*M]
    return y

### For small lengths

Comparision of MDCT over 16 samples

In [3]:
nsamples = 16
print(f"MDCT: {nsamples} samples.")
x = np.arange(nsamples)
%timeit mdctn.mdct(x, dct_type=4, norm='ortho')
%timeit mdct4(x)

print(f"IMDCT: {nsamples} samples.")
y = mdctn.mdct(x)
%timeit mdctn.imdct(y, dct_type=4, norm='ortho')
%timeit imdct4(y)

MDCT: 16 samples.
16 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
54.2 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
IMDCT: 16 samples.
11.9 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
37.8 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Medium

Comparision of MDCT over 1200 samples

In [4]:
nsamples = 1200
print(f"MDCT: {nsamples} samples.")
x = np.arange(nsamples)
%timeit mdctn.mdct(x, dct_type=4, norm='ortho')
%timeit mdct4(x)

print(f"IMDCT: {nsamples} samples.")
y = mdctn.mdct(x)
%timeit mdctn.imdct(y, dct_type=4, norm='ortho')
%timeit imdct4(y)




MDCT: 1200 samples.
25.7 µs ± 694 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
79.9 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
IMDCT: 1200 samples.
16.6 µs ± 77.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
64.9 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Huge

Comparision of MDCT over 24000 samples

In [5]:
nsamples = 24000
print(f"MDCT: {nsamples} samples.")
x = np.arange(nsamples)
%timeit mdctn.mdct(x, dct_type=4, norm='ortho')
%timeit mdct4(x)

print(f"IMDCT: {nsamples} samples.")
y = mdctn.mdct(x)
%timeit mdctn.imdct(y, dct_type=4, norm='ortho')
%timeit imdct4(y)

MDCT: 24000 samples.
106 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
450 µs ± 7.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
IMDCT: 24000 samples.
96.2 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
484 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
